# Methods


In [ ]:
# dependencies
import numpy as np
import matplotlib.pyplot as plt
import random as rd

In [ ]:
def foil(n, poly1, poly2, mod): # foils two polynomials

  nterms = 2*n - 1

  output = np.zeros(nterms)

  for i in range(n):
    temp = poly2.copy()
    temp = temp * poly1[i]
    padded_array = np.pad(temp, (i, nterms - n - i), 'constant', constant_values=0)
    output += padded_array

  if mod > 0:
    return np.mod(output, mod)

  return output

In [ ]:
def evaluate(poly, x, mod): # evaluates a polynomial at a given point

  total = 0

  for i in range(len(poly)):
    total += poly[i] * x**i

  return total % mod if mod > 0 else total


In [ ]:
def interpolate(input, n, mod): # generates the polynomial that fits n points

  size = len(input)
  laplace = np.zeros(2*n - 1);

  count = 0

  for i in range(size):

    if np.isnan(input[i]):
      continue

    nterms = 2
    f = np.array([1, 0])

    for j in range(size): # build laplace

      if np.isnan(input[j]):
        continue

      if i==j:
        continue

      temp = np.array([-1 * j, 1]) # zero at this value
      temp = np.pad(temp, (0, nterms - 2), 'constant', constant_values=0)
      #print(f)
      f = foil(nterms, f, temp, mod)

      if nterms >= n: # we only need n - 1 terms
        break
      nterms += 1;

    f = (f * input[i]) / evaluate(f, i, mod) # f[i] = input[i]
    print("polynomial for f(", i, ") = ", input[i], ": ", f)

    laplace = laplace + f

    count += 1
    if count >= n: # we only need n number of points
      break

  if mod > 0:
    return np.mod(laplace, mod)

  print("polyfit for input: ", laplace)

  return laplace


In [ ]:
def encode(input, laplace, n, k): # adds k new points given

  output = input.copy()

  for i in range(n, n+k):
    newval = evaluate(laplace, i, -1)
    output = np.append(output, newval)

  return output

In [ ]:
def add_noise(input, n, k): # strikes k points (error)

  for i in range(k):

    index = rd.randint(0, n-1);

    input[index] = np.nan;


In [ ]:
def decode(noisy_encoded, n, k): # finds original code

  output = np.zeros(n)

  laplace = interpolate(noisy_encoded, n, -1)

  for i in range(n):

    if np.isnan(noisy_encoded[i]):
      output[i] = evaluate(laplace, i, -1)
      continue

    output[i] = noisy_encoded[i]

  return output

# Simulation

In [ ]:
values = [1,2,3,5,2,7] # original signal
n = len(values) # original signal length
k = 6 # number of added values

In [ ]:
input = np.array(values)
laplace = interpolate(input, n, -1) # fits to polynomial of degree n-1

polynomial for f( 0 ) =  1 :  [ 1.         -2.28333333  1.875      -0.70833333  0.125      -0.00833333
 -0.         -0.         -0.         -0.         -0.        ]
polynomial for f( 1 ) =  2 :  [  0.          10.         -12.83333333   5.91666667  -1.16666667
   0.08333333   0.           0.           0.           0.
   0.        ]
polynomial for f( 2 ) =  3 :  [ -0.   -15.    26.75 -14.75   3.25  -0.25  -0.    -0.    -0.    -0.
  -0.  ]
polynomial for f( 3 ) =  5 :  [  0.          16.66666667 -32.5         20.41666667  -5.
   0.41666667   0.           0.           0.           0.
   0.        ]
polynomial for f( 4 ) =  2 :  [-0.         -2.5         5.08333333 -3.41666667  0.91666667 -0.08333333
 -0.         -0.         -0.         -0.         -0.        ]
polynomial for f( 5 ) =  7 :  [ 0.          1.4        -2.91666667  2.04166667 -0.58333333  0.05833333
  0.          0.          0.          0.          0.        ]
polyfit for input:  [  1.           8.28333333 -14.54166667   9.5  

In [ ]:
encoded = encode(input, laplace, n, k)
print(encoded)

[1.000e+00 2.000e+00 3.000e+00 5.000e+00 2.000e+00 7.000e+00 7.800e+01
 3.440e+02 1.031e+03 2.488e+03 5.213e+03 9.879e+03]


In [ ]:
add_noise(encoded, n, k)
print(encoded)

[1.000e+00       nan       nan       nan       nan       nan 7.800e+01
 3.440e+02 1.031e+03 2.488e+03 5.213e+03 9.879e+03]


In [ ]:
decoded = decode(encoded, n, k)

print("\ndecoded message: ",decoded)

polynomial for f( 0 ) =  1.0 :  [ 1.00000000e+00 -6.45634921e-01  1.65343915e-01 -2.09986772e-02
  1.32275132e-03 -3.30687831e-05 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
polynomial for f( 6 ) =  77.99999999999977 :  [ 0.00000000e+00  2.73000000e+03 -1.30758333e+03  2.33458333e+02
 -1.84166667e+01  5.41666667e-01  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00]
polynomial for f( 7 ) =  344.0 :  [-0.00000000e+00 -3.53828571e+04  1.77897143e+04 -3.30895238e+03
  2.70285714e+02 -8.19047619e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
polynomial for f( 8 ) =  1031.0 :  [ 0.00000000e+00  1.21786875e+05 -6.34065000e+04  1.22109062e+04
 -1.03100000e+03  3.22187500e+01  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00]
polynomial for f( 9 ) =  2488.000000000002 :  [-0.00000000e+00 -1.54808889e+05  8.27490370e+04 -1.64023704e+04
  1.42829630e+03 -4.60